### Linear Regression - High Level

In [6]:
# 필요한 라이브러리 호출
import torch
import torch.nn.functional as F
from torch.nn import Linear, MSELoss, Sequential
from torch.optim import Adam
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [42]:
# Linear class 직접 구현해보기
class MyLinear:
    def __init__(self, in_features, out_features):
        self.weight = torch.empty([in_features, out_features], requires_grad=True)
        self.bias = torch.empty([out_features], requires_grad=True)
        torch.nn.init.uniform_(self.weight)
        torch.nn.init.uniform_(self.bias)

    def parameters(self):
        return [self.weight, self.bias]
    
    def forward(self, x):
        pred = self.weight * x + self.bias
        return pred
    
    def __call__(self, x):
        pred = self.weight * x + self.bias
        return pred

my_linear = MyLinear(1, 1)
print(my_linear.parameters())
print(my_linear.forward(10))
print(my_linear(10)) # __call__

[tensor([[0.5095]], requires_grad=True), tensor([0.2348], requires_grad=True)]
tensor([[5.3302]], grad_fn=<AddBackward0>)
tensor([[5.3302]], grad_fn=<AddBackward0>)


In [11]:
# csv 파일 불러오기
df = pd.read_csv('data/cars.csv', index_col='Unnamed: 0')
df.head()

,speed,dist
1,4,2
2,4,10
3,7,4
4,7,22
5,8,16


In [27]:
# 속력과 제동거리를 행렬로 가져오기
x_data = df[['speed']].values
y_data = df[['dist']].values

In [28]:
# 텐서 노드 만들기
x = torch.FloatTensor(x_data)
y = torch.FloatTensor(y_data)

In [29]:
# Linear 클래스 생성
linear = Linear(1, 1)
print(list(linear.parameters())) # 랜덤으로 주어진 w와 b 값
print(linear.forward(torch.tensor([[10]], dtype=torch.float32)))
print(linear(torch.tensor([[10]], dtype=torch.float32))) # 예측값

[Parameter containing:
tensor([[0.3898]], requires_grad=True), Parameter containing:
tensor([0.2603], requires_grad=True)]
tensor([[4.1587]], grad_fn=<AddmmBackward>)
tensor([[4.1587]], grad_fn=<AddmmBackward>)


In [34]:
# 학습 진행
loss_fn = MSELoss()
optimizer = Adam(linear.parameters(), lr=0.1) # 미분 대상 w, b

for epoch in range(2000):
    optimizer.zero_grad() # 미분 초기화
    hx = linear.forward(x) # linear(x)과 같음
    c = loss_fn(hx, y) # 예측값과 실제값 / loss_fn.forward(hx, y)와 같음
    c.backward() # 미분 진행
    optimizer.step()
    print(f'{epoch} - cost : {c.item()}')

0 - cost : 11166.6640625
1 - cost : 10809.587890625
2 - cost : 10458.599609375
3 - cost : 10113.806640625
4 - cost : 9775.2978515625
5 - cost : 9443.1689453125
6 - cost : 9117.5029296875
7 - cost : 8798.3740234375
8 - cost : 8485.8583984375
9 - cost : 8180.01708984375
10 - cost : 7880.90478515625
11 - cost : 7588.5732421875
12 - cost : 7303.060546875
13 - cost : 7024.40234375
14 - cost : 6752.62060546875
15 - cost : 6487.73291015625
16 - cost : 6229.75
17 - cost : 5978.6689453125
18 - cost : 5734.482421875
19 - cost : 5497.1748046875
20 - cost : 5266.72314453125
21 - cost : 5043.09423828125
22 - cost : 4826.24755859375
23 - cost : 4616.1376953125
24 - cost : 4412.7080078125
25 - cost : 4215.89794921875
26 - cost : 4025.638671875
27 - cost : 3841.8544921875
28 - cost : 3664.464599609375
29 - cost : 3493.38134765625
30 - cost : 3328.510986328125
31 - cost : 3169.756591796875
32 - cost : 3017.01220703125
33 - cost : 2870.171630859375
34 - cost : 2729.120849609375
35 - cost : 2593.74536132

In [37]:
# 학습 후 w 값 확인
linear.weight

Parameter containing:
tensor([[4.0022]], requires_grad=True)

In [38]:
# 학습 후 b 값 확인
linear.bias

Parameter containing:
tensor([-18.7699], requires_grad=True)

In [41]:
# speed가 15, 20인 경우 dist 예측값 구하기 (행렬로 입력)
linear.forward(torch.FloatTensor([[15], [20]]))
# linear(torch.FloatTensor([[15], [20]]))

tensor([[41.2625],
        [61.2733]], grad_fn=<AddmmBackward>)

### 두 번째

In [54]:
model = Sequential() # Linear들이 쌓이게 됨 (DL)
model.add_module('linear1', Linear(1, 1))

lossfn = MSELoss()
optimizer = Adam(model.parameters(), lr=0.1)

for epoch in range(2000):
    optimizer.zero_grad() # 미분 초기화
    hx = model(x) # 모델의 x : 전처리, add_models의 forward 호출하여 linear(x)와 같음
    cost = loss_fn(hx, y)
    cost.backward() # 미분 진행
    optimizer.step()
    print(f'{epoch} - cost : {cost.item()}')

0 - cost : 1441.38720703125
1 - cost : 1326.5347900390625
2 - cost : 1217.7872314453125
3 - cost : 1115.23974609375
4 - cost : 1018.9681396484375
5 - cost : 929.0264282226562
6 - cost : 845.4432983398438
7 - cost : 768.2210693359375
8 - cost : 697.3326416015625
9 - cost : 632.7188110351562
10 - cost : 574.2869873046875
11 - cost : 521.9089965820312
12 - cost : 475.4198303222656
13 - cost : 434.616943359375
14 - cost : 399.2603454589844
15 - cost : 369.0732421875
16 - cost : 343.74395751953125
17 - cost : 322.92864990234375
18 - cost : 306.2552795410156
19 - cost : 293.32855224609375
20 - cost : 283.7361755371094
21 - cost : 277.0553894042969
22 - cost : 272.86090087890625
23 - cost : 270.7327880859375
24 - cost : 270.2643127441406
25 - cost : 271.07000732421875
26 - cost : 272.79229736328125
27 - cost : 275.107666015625
28 - cost : 277.7315368652344
29 - cost : 280.4212646484375
30 - cost : 282.9778137207031
31 - cost : 285.2461853027344
32 - cost : 287.1136169433594
33 - cost : 288.50

In [55]:
# 학습 후 w 값 확인
linear.weight

Parameter containing:
tensor([[4.0022]], requires_grad=True)

In [56]:
# 학습 후 b 값 확인
linear.bias

Parameter containing:
tensor([-18.7699], requires_grad=True)

In [57]:
# speed가 20인 경우 dist 예측값 구하기
model(torch.tensor([[20]], dtype=torch.float32))
# model[0](torch.tensor([[20]], dtype=torch.float32))

tensor([[61.0688]], grad_fn=<AddmmBackward>)